# Welcome

I'll add text here to help newcomers get started.

Basic things:
- you can run cells using "Shift+Enter"
- each cell runs on its own and they cannot run in parallel within the same notebook
- the variables are shared (i.e., global) between cells
- you can add text between cells to provide context; text uses Markdown formatting and can be 'plotted' using Shift+Enter
- I find it easier to add info on each cell in comments and use the Markdown text fields to add sections to the notebook to faciliate navigation (see table of contents in the left)

In [ ]:
# Install packages that might not be available at the Colab virtual machine. Feel free to add more if needed.
! pip install -U selenium
! apt-get update 
! apt install chromium-chromedriver

In [ ]:
# Mount YOUR google drive. You'll need to "Add shortcut to Drive" for our shared folder for it to show up here.
# Use the URL shown below in the output to authorize this Colab session to access you GDrive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# I usually import generally used packages at the beginning, so I suggest adding the ones we need to this cell as the notebook is developed
import pandas as pd
import time

# Scraping SFPID

In [ ]:
# these are packages specific to the scraping application, so I'd only import them within this section
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [ ]:
# User inputs 0:95386 (user defined)
start = 20651
end = 20654

# Paths
DATA_PATH = '/content/drive/MyDrive/Alameda Recovery/Datasets/input_searchfield2.xlsx'
EXPORT_PATH = '/content/drive/MyDrive/Alameda Recovery/Datasets/Scrape_Output/test_' \
              + str(start) + '_' + str(end) + '.txt'

# Import Vector of Search Field Inputs
data = pd.read_excel(DATA_PATH)
target = data[start:end+1]

# Selenium Webdriver Browser Access
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)
driver.get('https://sfplanninggis.org/pim/')


# Functions
def mapDisable():
    global driver
    map = driver.find_element_by_id("ReportMap")
    driver.execute_script("""
     var map = arguments[0];
     map.parentNode.removeChild(map);
     """, map)
    return map

def getFirst():
    global driver
    global target
    global EXPORT_PATH
    global start

    # Send Search Request
    PID = target['Parcel ID'].iloc[0]
    search = driver.find_element_by_id("addressInput")
    search.send_keys(PID)
    WebDriverWait(driver, 2).until(expected_conditions.visibility_of_element_located((By.XPATH, '//div[@id="Search-icon"]')))
    driver.find_element_by_id("Search-icon").click()

    # Wait for element to load
    status = driver.find_element_by_id("CensusTract").is_displayed()
    timeout = time.time() + 300
    while not status:
        status = driver.find_element_by_id("CensusTract").is_displayed()
        if time.time() > timeout:
            print('Does Not Exist, Attempting Alternative Path')
            print(start)
            break
        time.sleep(2)

    # Flag for error
    try:
        modal = driver.find_element_by_id("modalContentSmall")
        modal.find_element_by_xpath('.//table[1]/tbody[1]/tr[1]/td[1]/div').click()
        status = driver.find_element_by_id("CensusTract").is_displayed()
        timeout = time.time() + 300
        while not status:
            status = driver.find_element_by_id("CensusTract").is_displayed()
            if time.time() > timeout:
                print('Broken, No Fix')
                print(start)
                break
            time.sleep(2)

        # Get census tract
        census = driver.find_element_by_id("CensusTract")
        tract = census.find_element_by_xpath('.//div').text
        index = tract.rfind(" ")
        tractNum = tract[index + 1:]

        # Get sale date & price
        assess = driver.find_element_by_link_text("Assessor Summary").click()
        time.sleep(1)
        sale = driver.find_element_by_id("modalContent")
        try:
            date = sale.find_element_by_xpath('.//div[4]/div[1]/div[6]/div[2]').text
            date = date.translate({ord('-'): 'NaN'})
            price = sale.find_element_by_xpath('.//div[4]/div[1]/div[7]/div[2]').text
            price = price.translate({ord('$'): None})
            price = price.translate({ord('-'): 'NaN'})
        except:
            date = 'NaN'
            price = 'NaN'
        finally:
            # Close assessor report window & clear search field
            report = driver.find_element_by_id("PIMModal")
            report.find_element_by_xpath('.//div[1]/div[1]/div[1]/button').click()
            search = driver.find_element_by_id("addressInputSmall")
            search.clear()

            # Create Dataframe & Export
            df = pd.DataFrame(data=[start, PID, date, price, tractNum],
                              index=['i', 'Block Lot', 'Sale Date', 'Sale Price', 'Census Tract', ]).transpose()
            df.to_csv(EXPORT_PATH, header=None, sep='\t', mode='a')
            print(df)

    # Normal Execution
    except:
        # Get census tract
        census = driver.find_element_by_id("CensusTract")
        tract = census.find_element_by_xpath('.//div').text
        index = tract.rfind(" ")
        tractNum = tract[index + 1:]

        # Get sale date & price
        assess = driver.find_element_by_link_text("Assessor Summary").click()
        time.sleep(1)
        sale = driver.find_element_by_id("modalContent")
        try:
            date = sale.find_element_by_xpath('.//div[4]/div[1]/div[6]/div[2]').text
            date = date.translate({ord('-'): 'NaN'})
            price = sale.find_element_by_xpath('.//div[4]/div[1]/div[7]/div[2]').text
            price = price.translate({ord('$'): None})
            price = price.translate({ord('-'): 'NaN'})
        except:
            date = 'NaN'
            price = 'NaN'
        finally:
            # Close assessor report window & clear search field
            report = driver.find_element_by_id("PIMModal")
            report.find_element_by_xpath('.//div[1]/div[1]/div[1]/button').click()
            search = driver.find_element_by_id("addressInputSmall")
            search.clear()

            # Create Dataframe & Export
            df = pd.DataFrame(data=[start, PID, date, price, tractNum],
                              index=['i', 'Block Lot', 'Sale Date', 'Sale Price', 'Census Tract', ]).transpose()
            df.to_csv(EXPORT_PATH, header=None, sep='\t', mode='a')
            print(df)

def getLoop():
    global driver
    global target
    global EXPORT_PATH
    global start

    for i in range(len(target) - 1):
        # Search ID
        PID = target['Parcel ID'].iloc[i + 1]
        search = driver.find_element_by_id("addressInputSmall")
        search.send_keys(PID)
        WebDriverWait(driver, 2).until(expected_conditions.visibility_of_element_located((By.XPATH, '//div[@id="Search-icon2"]')))
        driver.find_element_by_id("Search-icon2").click()

        # Wait for element to load
        status = driver.find_element_by_id("CensusTract").is_displayed()
        timeout = time.time() + 300
        while not status:
            status = driver.find_element_by_id("CensusTract").is_displayed()
            if time.time() > timeout:
                print('Does Not Exist, Attempting Alternative Path')
                print(start+i+1)
                break
            time.sleep(2)

        # Flag for error
        try:
            modal = driver.find_element_by_id("modalContentSmall")
            modal.find_element_by_xpath('.//table[1]/tbody[1]/tr[1]/td[1]/div').click()
            status = driver.find_element_by_id("CensusTract").is_displayed()
            timeout = time.time() + 300
            while not status:
                status = driver.find_element_by_id("CensusTract").is_displayed()
                if time.time() > timeout:
                    print('Broken, No Fix')
                    print(start+1+i)
                    break
                time.sleep(2)

            # Get census tract
            census = driver.find_element_by_id("CensusTract") 
            tract = census.find_element_by_xpath('.//div').text
            index = tract.rfind(" ")
            tractNum = tract[index + 1:]

            # Get sale date & price
            driver.find_element_by_link_text("Assessor Summary").click()
            time.sleep(1)
            sale = driver.find_element_by_id("modalContent")
            try:
                date = sale.find_element_by_xpath('.//div[4]/div[1]/div[6]/div[2]').text
                date = date.translate({ord('-'): 'NaN'})
                price = sale.find_element_by_xpath('.//div[4]/div[1]/div[7]/div[2]').text
                price = price.translate({ord('$'): None})
                price = price.translate({ord('-'): 'NaN'})
            except:
                date = 'NaN'
                price = 'NaN'
            finally:
                # Close assessor report window & clear search field
                report = driver.find_element_by_id("PIMModal")
                report.find_element_by_xpath('.//div[1]/div[1]/div[1]/button').click()
                search = driver.find_element_by_id("addressInputSmall")
                search.clear()

                # Write to csv every iteration
                x = i + 1 + start
                df = pd.DataFrame(data=[x, PID, date, price, tractNum],
                                  index=['i', 'Block Lot', 'Sale Date', 'Sale Price', 'Census Tract', ]).transpose()
                df.to_csv(EXPORT_PATH, header=None, sep='\t', mode='a')

                if i % 10 == 0:
                    print(df)

        # Normal Execution
        except:
            # Get census tract
            census = driver.find_element_by_id("CensusTract") 
            tract = census.find_element_by_xpath('.//div').text
            index = tract.rfind(" ")
            tractNum = tract[index + 1:]

            # Get sale date & price
            driver.find_element_by_link_text("Assessor Summary").click()
            time.sleep(1)
            sale = driver.find_element_by_id("modalContent")
            try:
                date = sale.find_element_by_xpath('.//div[4]/div[1]/div[6]/div[2]').text
                date = date.translate({ord('-'): 'NaN'})
                price = sale.find_element_by_xpath('.//div[4]/div[1]/div[7]/div[2]').text
                price = price.translate({ord('$'): None})
                price = price.translate({ord('-'): 'NaN'})
            except:
                date = 'NaN'
                price = 'NaN'
            finally:
                # Close assessor report window & clear search field
                report = driver.find_element_by_id("PIMModal")
                report.find_element_by_xpath('.//div[1]/div[1]/div[1]/button').click()
                search = driver.find_element_by_id("addressInputSmall")
                search.clear()

                # Write to csv every iteration
                x = i + 1 + start
                df = pd.DataFrame(data=[x, PID, date, price, tractNum],
                                  index=['i', 'Block Lot', 'Sale Date', 'Sale Price', 'Census Tract', ]).transpose()
                df.to_csv(EXPORT_PATH, header=None, sep='\t', mode='a')

                if i % 10 == 0:
                    print(df)


# Scrape
getFirst()  # First Instance on Landing
mapDisable()  # Disable Map Element
getLoop()  # Loop Subsequent Instances
driver.close()  # Terminate Selenium Webdriver





# Misc, examples